# Sentence Classification using LSTM and Pretrained Word2Vec

We will train and test sentence classification using LSTM, and Pretrained Word2Vec.  
You can find visualization of our code below.

The most benefits from Pretrained Word Embedding is<br>even unseen words during traning can be predicted well, since pretrained word embedding already trained with larger data set than your train data.

for example, below example also can be predicted well, even "this", "best", "show" were not in the train data.<br> Since "this" is similar to "it", "best" is similar to "good" and "show" is similar to "movie" in pretrained word embedding vector. 

We will use LSTM, so we can generate sentence vector with sequence of word embedding.<br>LSTM is advanced RNN which is powerful on long sequence input.

In [7]:
!pip install tensorflow_hub 

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [6]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")
https://tfhub.dev/google/Wiki-words-250/2

OSError: SavedModel file does not exist at: C:\Users\nilay\AppData\Local\Temp\tfhub_modules\22bffebf9723377261c6cd19440f9601d0ec68d6\{saved_model.pbtxt|saved_model.pb}

In [ ]:
embed(["jump"])

In [5]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['review'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['sentiment'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

# Preprocess (encode text to number)

In [6]:
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
df = pd.DataFrame(movie_reviews_train)

# max_length is used for max sequence of input
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

# Build Model

In [7]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [9]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.6949 - accuracy: 0.7500
Epoch 2/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6850 - accuracy: 0.7500
Epoch 3/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6752 - accuracy: 0.7500
Epoch 4/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6655 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6557 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6457 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6356 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6252 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6145 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6034 - accuracy: 1.0000
Epoch 11/50
1/1 [=============

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                36224     
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


# Test
your model can predict correctly even for unseen words from training.  
This is the most benefit of using pretrained word embedding.  
Why? pretrained embedding will encode [better], [nice] to similar vector of [best]  
even if these words were not in train.  
therefore, the input vector to RNN is similar, so correct answers for even these unseen words.

In [11]:
"""
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
"""
movie_reviews_test = [
         {'review': 'it is better movie', 'sentiment': 'positive'},
         {'review': 'i suggest you see this movie', 'sentiment': 'positive'},
         {'review': 'it was just throwing 20 dollars away', 'sentiment': 'negative'},
         {'review': 'worse than any show', 'sentiment': 'negative'},
         {'review': 'nice movie, so love it', 'sentiment': 'positive'},
         {'review': 'terrible', 'sentiment': 'negative'}
    ]
test_df = pd.DataFrame(movie_reviews_test)

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 - 0s - loss: 0.1724 - accuracy: 1.0000 - 419ms/epoch - 419ms/step
Test score: 0.17236685752868652
Test accuracy: 1.0
